In [1]:
#Import the necessary packages
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Create the base url  
base_url = 'https://www.realtytrac.com/'

#Create a search for the LA area
la_search = 'mapsearch/sold/ca/los-angeles-county/'

In [3]:
#Grab the website content and store it in a BeautifulSoup object
agent = {"User-Agent":'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
cookies = {"cookie":"_ga=GA1.2.744553470.1600989246; _hjid=78f19d8b-c118-4bf1-9ceb-1a4d012a43a6; __qca=P0-1183409810-1600989246544; _hjTLDTest=1; _gid=GA1.2.1264249020.1601434220; _hjAbsoluteSessionInProgress=0; _gali=pageContent"}
response = requests.get(base_url+la_search, headers=agent, cookies=cookies)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
houses = soup.find_all('div', class_="house alt clearfix")

#Number of pages to run the scraper through
num_pages = 100

print(f'You\'ll scrape {len(houses)*num_pages} houses')

You'll scrape 1000 houses


In [5]:
absolute_links = []

for i in range(num_pages):
    page = soup.find_all('div', class_="house alt clearfix")
    
    for house in page:
        absolute_links.append(base_url+house.find('a')['href'])
    
    next_link = soup.find('a', class_='next')['href']
    response = requests.get(base_url+next_link, headers=agent, cookies=cookies)
    soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
#Create the function that extracts the attributes for each house(link)

df = pd.DataFrame()

def find_details(house):
    
    try:
        lat = house.find('meta', property='og:latitude')['content']
    except:
        lat=None
    try:
        long = house.find('meta', property='og:longitude')['content'] 
    except:
        long=None
    try:    
        address = house.find('span', class_='address heading').text 
    except:
        address=None
    
    #Property Details
    details = house.find('ul', class_='detail-list')
    add_details = details.find_all('li')
    
    try:
        prop_type = add_details[0].find('span', itemprop='name').text
    except:
        prop_type=None
    try:
        rooms = add_details[1].find('span', itemprop='description').text
    except:
        rooms=None
    try:
        home_size_p = details.find('span', itemprop='name', text='Home Size').parent
        home_size = home_size_p.find('span', itemprop='value').text
    except:
        home_size=''
    try:                                                              
        home_unit = home_size_p.find('span', itemprop='unitText').text
    except:
        home_unit=''
    try:
        lot_size_p = details.find('span', itemprop='name', text='Lot Size').parent
        lot_size = lot_size_p.find('span', itemprop='value').text
    except:
        lot_size=''
    try:                                                             
        lot_unit = lot_size_p.find('span', itemprop='unitText').text
    except:
        lot_unit=''
    try:
        year_built_p = details.find('span', itemprop='description', text='Built in').parent
        year_built = year_built_p.find('span', itemprop='value').text
    except:
        year_built=None
    try:
        parcel_p = details.find('span', itemprop='name', text='Parcel Number').parent
        parcel = parcel_p.find('span', itemprop='propertyID').text
    except:
        parcel=None
    try:
        realty_p = details.find('span', itemprop='name', text='RealtyTrac Property ID').parent
        realty = realty_p.find('span', itemprop='value').text
    except:
        realty=None
    try:
        county_p = details.find('span', itemprop='name', text='County').parent
        county = county_p.find('span', itemprop='description').text
    except:
        county=None
    try:
        sub_p = details.find('span', itemprop='name', text='Subdivision').parent
        sub = sub_p.find('span', itemprop='value').text
    except:
        sub=None
    try:
        census_p = details.find('span', itemprop='name', text='Census').parent
        census = census_p.find('span', itemprop='value').text
    except:
        census=None
    try:
        tract_p = details.find('span', itemprop='name', text='Tract').parent
        tract = tract_p.find('span', itemprop='value').text
    except:
        tract=None
    try:
        zoning = details.find_all('li')[-2].text
    except:
        zoning=None
    try:                                                             
        lot_p = details.find('span', itemprop='name', text='Lot').parent
        lot = lot_p.find('span', itemprop='value').text
    except:
        lot = None
    
    
    #Sales History
    sales = house.find('section', id='occupancy-link') 
    
    try:                                                              
        sale_price = sales.find('label', attrs={'for':'PurchasePrice'}).parent.parent.find('td', class_='col2').text            
    except:
        sale_price=None
    try:
        date = sales.find('label', attrs={'for':'PurchaseDate'}).parent.parent.find('td', class_='col2').text
    except:
        date=None
    
    #Estimated Value
    est = house.find('div', class_='rate-row access')
    
    try:                                                              
        estimate = est.find('strong', class_='price').text
    except:
        estimate= None
    
    #Home Disclosures
    disc = house.find('div', id="collapsePropertyHomeDisclosuresArea")
    
    #Need help rendering the javascript portion
    
    #sex_offenders = disc.find('li', class_='offenders alert').find('span', class_='hd-label').text
    #crime_index = disc.find('li', class_='crime concern').find('span', class_='hd-label').text
    #env_hazards = disc.find('li', class_='enviornmental alert').find('span', class_='hd-label').text
    #natural_disasters = disc.find('li', class_='disaster alert').find('span', class_='hd-label').text
    #school_index = disc.find('li', class_='schools alert').find('span', class_='hd-label').text
    
    #Create the df to be returned
    try:
        url = house.find('meta', property='og:url')['content']
    except:
        url=None
    
    df = pd.DataFrame({
                       'Latitude': [lat],
                       'Longitude': [long],
                       'Address':[address],
                       'Property_type': [prop_type],
                       'Rooms': [rooms],
                       'Home_size': [home_size+home_unit],
                       'Lot Size': [lot_size+lot_unit],
                       'Year_built': [year_built],
                       'Parcel_number': [parcel],
                       'RealtyID': [realty],
                       'County': [county],
                       'Subdivision': [sub],
                       'Census': [census],
                       'Tract': [tract],
                       'Lot': [lot],
                       'Zoning': [zoning],
                       'Date': [date],
                       'Sale_price': [sale_price],
                       'Estimated Value': [estimate],
                       #'Sex_Offenders': [sex_offenders],
                       #'Crime_index': [crime_index],
                       #'Enviornmental_Hazards': [env_hazards],
                       #'School Quality': [school_index]
                       'url': [url]
                      })
    
    return df

In [7]:
#Spot check 
house_detail = requests.get(absolute_links[24])
soup = BeautifulSoup(house_detail.content, 'html.parser')
df = df.append(find_details(soup), ignore_index=True)
df

,Latitude,Longitude,Address,Property_type,Rooms,Home_size,Lot Size,Year_built,Parcel_number,RealtyID,County,Subdivision,Census,Tract,Lot,Zoning,Date,Sale_price,Estimated Value,url
0,34.574381,-118.037508,38058 Lido Dr,Single Family Residence,"4 beds, 3 baths","1,979 sqft","7,036 sqft",1989,3023054044,1110738665,Los Angeles,45073,1,910713,45,\r\nZoning:\r\n\r\nPDA2*\r\n\t\t,09/16/2020,"$350,000","$334,000",https://www.realtytrac.com/property/ca/palmdal...


In [8]:
#Loop through the houses and grab all of the data

for i in range(len(absolute_links)):
    house_detail = requests.get(absolute_links[i])
    soup = BeautifulSoup(house_detail.content, 'html.parser')
    df = df.append(find_details(soup), ignore_index=True)
    
    if (i+1)%100 ==0:
        df.to_csv('Data/house_data_scraped.csv', index=False)
        print(f'Scraped {df.shape[0]-1} properties')
df.head()

Scraped 100 properties
Scraped 200 properties
Scraped 300 properties
Scraped 400 properties
Scraped 500 properties
Scraped 600 properties
Scraped 700 properties
Scraped 800 properties
Scraped 900 properties
Scraped 1000 properties


,Latitude,Longitude,Address,Property_type,Rooms,Home_size,Lot Size,Year_built,Parcel_number,RealtyID,County,Subdivision,Census,Tract,Lot,Zoning,Date,Sale_price,Estimated Value,url
0,34.574381,-118.037508,38058 Lido Dr,Single Family Residence,"4 beds, 3 baths","1,979 sqft","7,036 sqft",1989,3023054044,1110738665,Los Angeles,45073,1,910713,45,\r\nZoning:\r\n\r\nPDA2*\r\n\t\t,09/16/2020,"$350,000","$334,000",https://www.realtytrac.com/property/ca/palmdal...
1,34.029588,-118.34438,4711 W 29th St,Single Family Residence,"2 beds, 1 bath","1,132 sqft","2,750 sqft",1927,5057008024,1110724102,Los Angeles,5780,5,220000,623,\r\nZoning:\r\n\r\nLAR2\r\n\t\t,09/16/2020,"$888,000","$794,000",https://www.realtytrac.com/property/ca/los-ang...
2,33.988016,-117.984728,15362 Del Prado Dr,Single Family Residence,"4 beds, 3 baths","2,462 sqft","10,250 sqft",1970,8290004038,1110725454,Los Angeles,29726,3,408503,10,\r\nZoning:\r\n\r\nLCRA1L\r\n\t\t,09/16/2020,"$890,000","$802,000",https://www.realtytrac.com/property/ca/haciend...
3,33.879723,-118.065015,16622 Shenandoah Ave,Condominium,"2 beds, 2 baths",904 sqft,"7,007 sqft",1973,7012006007,1110725503,Los Angeles,31077,1,554514,None,\r\nZoning:\r\n\r\nCERM\r\n\t\t,09/16/2020,"$420,000","$428,000",https://www.realtytrac.com/property/ca/cerrito...
4,34.051043,-118.254448,633 W 5th St,Office Building,None,30.9 acres,1.17 acres,1988,5151017028,1110725507,Los Angeles,MOTT,1,207502,6,\r\nZoning:\r\n\r\nLAC5\r\n\t\t,09/16/2020,"$430,000,000",None,https://www.realtytrac.com/property/ca/los-ang...


In [9]:
#Save the dataframe to a csv file
df.to_csv('Data/house_data_scraped.csv', index=False)